# Skip notebook test
-----

# Running cuGraph using synthetic or stored data sets on various algorithms on single node multi GPU (SNMG) cluster


This notebook runs and times the execution of many of the cuGraph algorithms when run against data at multiple scales.

This notebook constains the RMAT data generator which allows the creation of graphs at various scales.  The notebook is configurable to run either on the RMAT data or on proved real-life datasets. Using the cuGraph Datasets API, many other datasets are available.


__IMPORTANT NOTE: The timings generated by this notebook are for informational purposes only. They do not represent publishable benchmarks but are instead meant to demonstrate loading a graph, running cuGraph algorithms and timing them.__

### Timing 

 This notebook produces run times for:
 - (1)	Each algorithm
 - (2)	Graph creation time for each data set

Each is run a configured number of times and the average run time is stored.

Since GPU memory is a precious resource, having a lot of temporary data laying around is avoided.  So once a graph is created, the raw data is dropped.  
 
__What is not timed__:  Generating the data with R-MAT</p>
__What is timed__:     (1) creating a Graph, (2) running the algorithm


### Algorithms

|        Algorithm        |  Type         | Undirected Graph | Directed Graph |   Notes
| ------------------------|---------------|------ | ------- |-------------
| WCC                     | Components    |   X   |         |
| Katz                    | Centrality    |   X   |         |
| Betweenness Centrality  | Centrality    |   X   |         | Estimated, k = 100
| K Truss                 | Community     |   X   |         |
| Louvain                 | Community     |   X   |         | Uses python-louvain for comparison
| Triangle Counting       | Community     |   X   |         |
| Core Number             | Core          |   X   |         |
| PageRank                | Link Analysis |       |    X    |
| Jaccard                 | Similarity    |   X   |         |one-hop over all connected nodes instead of 2-hop default
| BFS                     | Traversal     |   X   |         | No depth limit
| SSSP                    | Traversal     |   X   |         |


### Test Data
Data is generated using a Recursive MATrix (R-MAT) graph generation algorithm. 
The generator specifics are documented [here](https://docs.rapids.ai/api/cugraph/stable/api_docs/generator.html)



In [ ]:
mg=True
data_to_use = "Data Sets"
number_of_runs = 1

In [ ]:
# Test Data Sizes
# Here you can create an array of test data sizes.   Then set the "data" variable to the array you want
# the dictionary format is 'name' : scale


# These scales are used by R-MAT to determine the number of vertices/edges in the synthetic data graph.
data_full = {
    'RMAT Scale 12'   :  12,
    'RMAT Scale 14'  :   14,
    'RMAT Scale 16'  :   16,
    'RMAT Scale 18'  :   18,
    'RMAT Scale 20'  :   20,
    'RMAT Scale 22'  :   22,
}

# for quick testing
data_quick = {
   'RMAT Scale 9' : 9,
   'RMAT Scale 10' : 10,
   'RMAT Scale 11' : 11,
}

# for existing datasets
data_sets = {
    'netscience' : -1,
    'hollywood' : -1,
    'cit_patents' : -1,
    'email_Eu_core' : -1,
}


In [ ]:
# Which dataset is to be used

data = None

if data_to_use == "Data Sets":
    data = data_sets
if data_to_use == "Small Scale RMAT":
    data = data_quick
if data_to_use == "Large Scale RMAT":
    data = data_full
if data == None:
    raise Exception("Invalid data specification")

## Import Modules

In [ ]:
# system and other
import gc
import os
import importlib
import sys
from functools import partial
import time
import timeit
import pandas as pd
from collections import defaultdict

# rapids
import cugraph
import cugraph.datasets as ds

# liblibraries to setup dask cluster and client
from dask.distributed import Client
from dask_cuda import LocalCUDACluster
from cugraph.dask.comms import comms as Comms


# RMAT data generator
from cugraph.generators import rmat
from cugraph.structure import NumberMap

### Determine the scale of the test data
RMAT generates graph where the number of vertices is a power of 2 and the number of edges is based on an edge factor times the number vertices.

Since RMAT tends to generate about 50% isolated vertices, those vertices are dropped from the graph data.  Hence the number of vertices is closer to (2 ** scale) / 2


| Scale | Vertices (est) | Edges  |
| ------|----------------|--------|
| 10 | 512 | 16,384 | 
| 11 | 1,024 | 32,768| 
| 12 | 2,048 | 65,536| 
| 13 | 4,096 | 131,072| 
| 14 | 8,192 | 262,144| 
| 15 | 16,384 | 524,288 | 
| 16 | 32,768 | 1,048,576 | 
| 17 | 65,536 | 2,097,152 | 
| 18 | 131,072 | 4,194,304 | 
| 19 | 262,144 | 8,388,608 | 
| 20 | 524,288 | 16,777,216 | 
| 21 | 1,048,576 | 33,554,432 | 
| 22 | 2,097,152 | 67,108,864 | 
| 23 | 4,194,304 | 134,217,728 | 
| 24 | 8,388,608 | 268,435,456 | 
| 25 | 16,777,216 | 536,870,912 | 


### Generate data
The data is generated once for each graph only when doing random data.

In [ ]:
# Data generator 
#  The result is an edgelist of the size determined by the scale and edge factor
def generate_data(scale, edgefactor=16, mg=False):
    _gdf = rmat(
        scale,
        (2 ** scale) * edgefactor,
        0.57,
        0.19,
        0.19,
        42,
        clip_and_flip=False,
        scramble_vertex_ids=True,
        create_using=None,  # return edgelist instead of Graph instance
        mg=mg # determines whether generated data will be used on one or multiple GPUs
        )

    clean_coo = NumberMap.renumber(_gdf, src_col_names="src", dst_col_names="dst")[0]
    if mg:
        clean_coo.rename(columns={"renumbered_src": "src", "renumbered_dst": "dst"})
    else:
        clean_coo.rename(columns={"renumbered_src": "src", "renumbered_dst": "dst"}, inplace=True)

    print(f'Generated a dataframe of type {type(clean_coo)}, with {len(clean_coo)} edges')
    
    return clean_coo

In [ ]:
def format_times(run_time):
    if isinstance(run_time, (int, float)):
        if run_time > 0:
            return f"{run_time:.4g}"
        return 0.0
    return run_time

In [ ]:
import subprocess

def get_gpus():
    try:
        gpu_info = subprocess.check_output(
        ['nvidia-smi', '--query-gpu=name', '--format=csv,noheader'],
        encoding='utf-8'
        )
        gpus = [line.strip() for line in gpu_info.strip().split('\n') if line.strip()]
        gpu_count = len(gpus)
        return ''.join(set(gpus)), gpu_count
    except Exception as e:
        return "no_gpus", 0


In [ ]:
get_gpus()

## Create Graph functions
There are two types of graphs created:
* Directed Graphs - create_cu_directed_graph.
* Undirected Graphs - calls to create_cu_ugraph <- fully symmeterized

In [ ]:
# cuGraph
def create_cu_graph(_df, transpose=True, directed=False, mg=False):
    _g = cugraph.Graph(directed=directed)

    if mg:
        _g.from_dask_cudf_edgelist(_df, source="src", destination="dst", edge_attr=None)
        print("Created mg graph")
    else:
        _g.from_cudf_edgelist(_df,
                            source='src',
                            destination='dst',
                            edge_attr=None,
                            renumber=False,
                            store_transposed=transpose)
        print("Created single GPU graph")

    return _g

## Algorithm Execution

### 	Weakly Connected Components (WCC)

In [ ]:
def cu_wcc(_G, mg=False):
    if mg:
        _ = cugraph.dask.weakly_connected_components(_G)
    else:
        _ = cugraph.weakly_connected_components(_G)
    return

### Katz

In [ ]:

def cu_katz(_G, alpha, mg=False):
    if mg:
        _ = cugraph.dask.katz_centrality(_G, alpha)
    else:
        _ = cugraph.katz_centrality(_G, alpha)
    return


### Betweenness Centrality (BC)

In [ ]:

def cu_bc(_G, _k, mg=False):
    if mg:
        _ = cugraph.dask.betweenness_centrality(_G, k=_k)
    else:   
        _ = cugraph.betweenness_centrality(_G, k=_k)
    return

### Louvain

In [ ]:
def cu_louvain(_G, mg=False):
    if mg:
        _, modularity = cugraph.dask.louvain(_G)
    else:
        _,_ = cugraph.louvain(_G)
    return



### K-Truss

In [ ]:
def cu_ktruss(_G, k, mg=False):
    try:
        if mg:
            _ = cugraph.dask.ktruss_subgraph(_G,k)
        else:
            _ = cugraph.ktruss_subgraph(_G,k)
    except:
        print(f"K_Truss failed to run"),
        return -1.0
    return

### Triangle Counting

In [ ]:
def cu_tc(_G, mg=False):
    if mg:
        _ = cugraph.dask.triangle_count(_G)
    else:
        _ = cugraph.triangle_count(_G)
    return


### Core Number

In [ ]:
def cu_core_num(_G, mg=False):
    if mg:
        _ = cugraph.dask.core_number(_G)
    else:
        _ = cugraph.core_number(_G)
    return


### PageRank

In [ ]:
def cu_pagerank(_G, mg=False):
    if mg:
        _ = cugraph.dask.pagerank(_G)
    else:
        _ = cugraph.pagerank(_G)
    return


### Jaccard

In [ ]:
def cu_jaccard(_G, mg=False):
    edge_list = _G.view_edge_list()
    if mg:
        _ = cugraph.dask.jaccard(_G, vertex_pair = edge_list )
    else:
        _ = cugraph.jaccard(_G, vertex_pair = edge_list)
    return


### Breadth First Search (BFS)

In [ ]:
def cu_bfs(_G, seed=0, mg=False):
    if mg:
        _ = cugraph.dask.bfs(_G, seed)
    else:
        _ = cugraph.bfs(_G, seed)
    return


### Single Source Shortest Path (SSSP)

In [ ]:
def cu_sssp(_G, seed = 0, mg=False):
    
    # SSSP requires weighted graph
    if mg:
        if _G.weighted: 
            _ = cugraph.dask.sssp(_G, seed)
        else:
            _ = cugraph.dask.bfs(_G, seed)

    else:
        if _G.weighted:
            _ = cugraph.ssp(_G, seed)
        else:
            _ = cugraph.bfs(_G, seed)
    return


### Initialize multi-GPU environment
Before we get started, we need to set up a dask (local) cluster of workers to execute our work, and a client to coordinate and schedule work for that cluster.


In [ ]:
# Setup a local dask cluster of workers, and a client
cluster = LocalCUDACluster()
client = Client(cluster)
Comms.initialize(p2p=True)

### Run cuGraph algorithms for datasets a parameterized number of times and collect the average times
Takes in a mg parameter to determine if multiple GPU are used when available.

In [ ]:
def run_algorithms( dataset, scale, mg):

    cugraph_algo_run_times = defaultdict()

    # generate data
    print("------------------------------")
    if (scale != -1):
        gdf = generate_data(scale, edgefactor=16, mg=mg)
    # gdf = gdf.repartition(gdf.npartitions * 3)
    else:
        current_set = getattr (ds, dataset)
        if mg == True:
            gdf = current_set.get_dask_edgelist(download=True)
        else:
            gdf = current_set.get_edgelist(download=True)

        print(type(gdf))
    # create cuGraph
    g_cu = create_cu_graph(gdf, mg=mg, transpose =True)

    # time graph creation
    result = timeit.Timer(lambda: create_cu_graph(gdf, mg=mg, transpose =True)).repeat(repeat=1, number=number_of_runs)
    tc = sum(result) / number_of_runs

    cugraph_graph_creation_time = tc
    del gdf

    # prep
    deg = g_cu.degree()
    if mg == True:
        deg_max = deg['degree'].max().compute()
    else:
        deg_max = deg['degree'].max()
    alpha = 1 / deg_max
    num_nodes = g_cu.number_of_vertices()
    del deg
    gc.collect()

    #-- WCC
    algorithm = "WCC"
    print(f"\t{algorithm}  ", end = '')
    result = timeit.Timer(lambda: cu_wcc(g_cu, mg=mg)).repeat(repeat=1, number=number_of_runs)
    tc = sum(result) / number_of_runs
    print("")
    
    cugraph_algo_run_times[algorithm] = tc

    #-- Katz
    algorithm = "Katz"
    print(f"\t{algorithm}  ", end = '')
    result = timeit.Timer(lambda: cu_katz(g_cu, alpha, mg=mg)).repeat(repeat=1, number=number_of_runs)
    tc =  sum(result) / number_of_runs
    print("")
    
    cugraph_algo_run_times[algorithm] = tc

    #-- K Truss
    algorithm = "K_Truss"
    print(f"\t{algorithm}  ", end = '')
    k = 5
    result = timeit.Timer(lambda: cu_ktruss(g_cu, k=k, mg=mg)).repeat(repeat=1, number=number_of_runs)
    tc =  sum(result) / number_of_runs
    print("")
    
    cugraph_algo_run_times[algorithm] = tc

    #-- BC
    algorithm = "BC"
    print(f"\t{algorithm}  ", end = '')
    k = 100
    if k > num_nodes:
        k = int(num_nodes)
    result = timeit.Timer(lambda: cu_bc(g_cu, k, mg=mg)).repeat(repeat=1, number=number_of_runs)
    tc =  sum(result) / number_of_runs
    print(" ")
    cugraph_algo_run_times[algorithm] = tc


    #-- Louvain
    algorithm = "Louvain"
    print(f"\t{algorithm}  ", end = '')
    result = timeit.Timer(lambda: cu_louvain(g_cu, mg=mg)).repeat(repeat=1, number=number_of_runs)
    tc =  sum(result) / number_of_runs

    print(" ")

    cugraph_algo_run_times[algorithm] = tc

    #-- TC
    algorithm = "TC"
    print(f"\t{algorithm}  ", end = '')
    result = timeit.Timer(lambda: cu_tc(g_cu, mg=mg)).repeat(repeat=1, number=number_of_runs)
    tc =  sum(result) / number_of_runs
    print(" ")
    
    cugraph_algo_run_times[algorithm] = tc

    #-- Core Number
    algorithm = "Core Number"
    print(f"\t{algorithm}  ", end = '')
    result = timeit.Timer(lambda: cu_core_num(g_cu, mg=mg)).repeat(repeat=1, number=number_of_runs)
    tc =  sum(result) / number_of_runs
    print(" ")

    cugraph_algo_run_times[algorithm] = tc

    #-- PageRank
    algorithm = "PageRank"
    print(f"\t{algorithm}  ", end = '')
    result = timeit.Timer(lambda: cu_pagerank(g_cu, mg=mg)).repeat(repeat=1, number=number_of_runs)
    tc =  sum(result) / number_of_runs
    print(" ")

    cugraph_algo_run_times[algorithm] = tc

    #-- Jaccard
    algorithm = "Jaccard"
    print(f"\t{algorithm}  ", end = '')
    result = timeit.Timer(lambda: cu_jaccard(g_cu, mg=mg)).repeat(repeat=1, number=number_of_runs)
    tc =  sum(result) / number_of_runs
    print(" ")

    cugraph_algo_run_times[algorithm] = tc

    # Seed for BFS and SSSP
    if mg == True:
        cu_seed = g_cu.nodes().compute().to_pandas().iloc[0]
    else:
        cu_seed = g_cu.nodes().to_pandas().iloc[0]

    #-- BFS
    algorithm = "BFS"
    print(f"\t{algorithm}  ", end = '')
    result = timeit.Timer(lambda: cu_bfs(g_cu, seed=cu_seed, mg=mg)).repeat(repeat=1, number=3)
    tc =  sum(result) / number_of_runs
    print(" ")

    cugraph_algo_run_times[algorithm] = tc

    #-- SSSP
    algorithm = "SSSP"
    print(f"\t{algorithm}  ", end = '')
    result = timeit.Timer(lambda: cu_sssp(g_cu, seed=cu_seed, mg=mg)).repeat(repeat=1, number=3)
    tc =  sum(result) / number_of_runs
    print(" ")

    cugraph_algo_run_times[algorithm] = tc

    del g_cu
    gc.collect()

    return cugraph_algo_run_times, cugraph_graph_creation_time


### cuGraph execution times for different algorithms
Run this with mg=True to use multiple GPUs

In [ ]:
cugraph_algo_run_times = defaultdict(defaultdict)
cugraph_graph_creation_time = defaultdict()
for dataset, scale in data.items():
    cugraph_algo_run_times[dataset], cugraph_graph_creation_time[dataset] = run_algorithms(dataset, scale, mg=mg )
gpu,count = get_gpus()

In [ ]:
cugraph_algo_run_times

In [ ]:
cugraph_graph_creation_time

In [ ]:
# cuGraph execution times for different algorithms
cugraph_run_times = pd.DataFrame()
for dataset in cugraph_algo_run_times.keys():
    temp_df = pd.DataFrame({'cuGraph': cugraph_algo_run_times[dataset]})
    temp_df.loc['Creation Time'] = cugraph_graph_creation_time[dataset]
    columns = [(dataset, '')]
    temp_df.columns = pd.MultiIndex.from_tuples(columns)
    cugraph_run_times = pd.concat([temp_df, cugraph_run_times], axis=1)

print(f'\n\t------cuGraph execution times for different algorithms-----mg={mg}\n')
print(cugraph_run_times)

Function to Capture the GPU info

In [ ]:
import subprocess

def get_gpus():
    try:
        gpu_info = subprocess.check_output(
        ['nvidia-smi', '--query-gpu=name', '--format-csv,noheader'],
        encoding='utf-8'
        )
        gpus = [line.strip() for line in output.strip().split('\n') if line.strip()]
        gpu_count = len(gpus)
        return gpus, gpu_count
    except Exception as e:
        return "no_gpus", 0

### Clean up multi-GPU environment

In [ ]:
Comms.destroy()
client.close()
cluster.close()

___
Copyright (c) 2020-2025, NVIDIA CORPORATION.

Licensed under the Apache License, Version 2.0 (the "License");  you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.
___